In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

import numpy as np
from torch import nn
import torch.nn.functional as F

from google.colab import drive

drive.mount('/content/drive')

In [0]:
!ls "/content/drive/My Drive/Colab Notebooks/data/"

labels.txt  reviews.txt


In [0]:
with open('/content/drive/My Drive/Colab Notebooks/data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('/content/drive/My Drive/Colab Notebooks/data/labels.txt', 'r') as f:
    labels = f.read()

In [0]:
print(reviews[:2000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [0]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [0]:
# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [0]:
words[:10]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the']

In [0]:
# feel free to use this import 
from collections import Counter
import operator

## Build a dictionary that maps words to integers
word_freq = Counter(words)
word_freq = sorted(word_freq.items(), key=operator.itemgetter(1), reverse=True)

int2vocab = dict(enumerate(map(lambda x: x[0], word_freq), start=1))

## Build a dictionary that maps words to integers
vocab_to_int = {ch: ii for ii, ch in int2vocab.items()}

In [0]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = [[vocab_to_int[word] for word in ch.split()] for ch in reviews_split]


In [0]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [0]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')

encoded_labels = np.array([1  if label.lower() == 'positive' else 0 for label in labels_split])

print(labels_split[:20])
print(encoded_labels[:20])

['positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative']
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


In [0]:
review_lens = Counter([len(r) for r in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [0]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
review_len_list = [len(x) for x in reviews_ints]
index = review_len_list.index(min(review_len_list))

del reviews_ints[index]
encoded_labels = np.delete(encoded_labels, index)

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


In [0]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## implement function
    features=[]
    for review in reviews_ints:
      if len(review) >= seq_length:
        features.append(review[:seq_length])
      else:
        padded_review = [0 for _ in range(seq_length - len(review))]
        padded_review.extend(review)
        features.append(padded_review)
    return np.array(features)

In [0]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30, :10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [0]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
from sklearn.model_selection import train_test_split

split_count = int(features.shape[0] * split_frac)

train_X, test_X1, train_Y, test_Y1 = train_test_split(features, encoded_labels, 
                                                    test_size=1-split_frac)

test_len = int(len(test_X1) * .5)
  
validation_X, validation_Y = test_X1[:test_len], test_Y1[:test_len]
test_X, test_Y = test_X1[test_len:], test_Y1[test_len:]
## print out the shapes of your resultant feature data

print(train_X.shape, train_Y.shape)
print(test_X.shape, test_Y.shape)
print(validation_X.shape, validation_Y.shape)

(20000, 200) (20000,)
(2500, 200) (2500,)
(2500, 200) (2500,)


In [0]:
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_Y))
valid_data = TensorDataset(torch.from_numpy(validation_X), torch.from_numpy(validation_Y))
test_data = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_Y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   0,    0,    0,  ...,    7,    3,  308],
        [  35, 3297,  921,  ...,  924,  479,   74],
        [1357, 3928, 2072,  ..., 2742,    6, 3502],
        ...,
        [   0,    0,    0,  ...,    3,  772,   56],
        [   1,  112,    8,  ..., 1801,    5,  141],
        [   0,    0,    0,  ...,    1, 3762, 6173]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 1])


In [0]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, 
                            num_layers=n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(p=drop_prob)
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.shape[0]
        x = self.embeds(x)
        r_out, hidden = self.lstm(x, hidden)
        self.dropout(r_out)
        r_out = r_out.contiguous().view(-1, self.hidden_dim)
        sig_out = torch.sigmoid(self.fc(r_out))
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        if train_on_gpu:
            hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda(), 
                      torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda())
        else:
            hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim), 
                      torch.zeros(self.n_layers, batch_size, self.hidden_dim))
        return hidden
        

In [0]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embeds): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)


In [0]:
lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [0]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.611672... Val Loss: 0.658402
Epoch: 1/4... Step: 200... Loss: 0.640246... Val Loss: 0.625697
Epoch: 1/4... Step: 300... Loss: 0.627909... Val Loss: 0.643314
Epoch: 1/4... Step: 400... Loss: 0.677242... Val Loss: 0.673572
Epoch: 2/4... Step: 500... Loss: 0.497949... Val Loss: 0.619903
Epoch: 2/4... Step: 600... Loss: 0.506340... Val Loss: 0.443043
Epoch: 2/4... Step: 700... Loss: 0.425031... Val Loss: 0.395482
Epoch: 2/4... Step: 800... Loss: 0.419974... Val Loss: 0.468853
Epoch: 3/4... Step: 900... Loss: 0.453529... Val Loss: 0.384042
Epoch: 3/4... Step: 1000... Loss: 0.294762... Val Loss: 0.366109
Epoch: 3/4... Step: 1100... Loss: 0.419041... Val Loss: 0.385828
Epoch: 3/4... Step: 1200... Loss: 0.297505... Val Loss: 0.354007
Epoch: 4/4... Step: 1300... Loss: 0.112567... Val Loss: 0.416388
Epoch: 4/4... Step: 1400... Loss: 0.321812... Val Loss: 0.375648
Epoch: 4/4... Step: 1500... Loss: 0.288905... Val Loss: 0.374635
Epoch: 4/4... Step: 1600... Loss: 

In [0]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.433
Test accuracy: 0.832


In [0]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [0]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    
    
    # print custom response based on whether test_review is pos/neg
    # get rid of punctuation
    test_review = test_review.lower() # lowercase, standardize
    all_text = ''.join([c for c in test_review if c not in punctuation])
    review_ints = [[vocab_to_int[word] for word in all_text.split()]]
    feature = pad_features(review_ints, sequence_length)
    print(feature.shape)
    batch_size = feature.shape[0]
    net.eval()
    h = net.init_hidden(batch_size)
    
    if train_on_gpu:
        feature = torch.from_numpy(feature).cuda()
    else:
        feature = torch.from_numpy(feature)
    output, _ = net(feature, h)
    review = ['Negative', 'Positive']
    
    output = torch.round(output)
    print(f'Review : {test_review}\nSentiment : {review[int(output.item())]}')

In [0]:
predict(net, test_review_neg)

(1, 200)
Review : the worst movie i have seen; acting was terrible and i want my money back. this movie had bad acting and the dialogue was slow.
Sentiment : Negative


In [0]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [0]:
# call function
# try negative and positive reviews!
seq_length=200
predict(net, test_review_pos, seq_length)

(1, 200)
Review : this movie had the best acting and the dialogue was so good. i loved it.
Sentiment : Positive
